In [64]:
from ib_async import *
import requests, pytz
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_market_calendars as mcal
from zoneinfo import ZoneInfo
import yfinance as yf
from data_and_research import ac

In [120]:
def get_last_full_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = pytz.timezone('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now(pytz.timezone('Europe/Berlin'))
    elif current_datetime.tzinfo is None:
        current_datetime = pytz.timezone('Europe/Berlin').localize(current_datetime)
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for the current day and the previous few days
    schedule = nyse.schedule(start_date=nyse_time.date() - timedelta(days=5), end_date=nyse_time.date())
    
    if not schedule.empty:
        # Get the last row of the schedule (should be today or the most recent trading day)
        last_day = schedule.iloc[-1]
        market_close = last_day['market_close'].tz_convert(nyse_tz)
        
        # If current time is before the market close of the last day in the schedule,
        # we need to look at the previous trading day
        if nyse_time < market_close:
            if len(schedule) > 1:
                return schedule.index[-2].date()
            else:
                # If there's only one day in the schedule, find the previous trading day
                previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
                return previous_trading_days[-1].date()
        else:
            # If current time is after market close, the last day in the schedule is the last full trading day
            return last_day.name.date()
    else:
        # If there's no schedule for the recent days (e.g., long weekend), 
        # find the last trading day
        previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
        return previous_trading_days[-1].date()

In [121]:
get_last_full_trading_day()

datetime.date(2024, 9, 30)

In [122]:
def get_current_or_next_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = pytz.timezone('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now(pytz.timezone('Europe/Berlin'))
    elif current_datetime.tzinfo is None:
        current_datetime = pytz.timezone('Europe/Berlin').localize(current_datetime)
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for today and the next few days
    schedule = nyse.schedule(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
    
    if not schedule.empty:
        today_schedule = schedule.loc[schedule.index.date == nyse_time.date()]
        if not today_schedule.empty:
            market_open = today_schedule.iloc[0]['market_open'].tz_convert(nyse_tz)
            market_close = today_schedule.iloc[0]['market_close'].tz_convert(nyse_tz)
            
            # If the current time is before market close, return today
            if nyse_time <= market_close:
                return nyse_time.date()
    
    # If we're here, it means today is not a trading day or the market has closed
    # Find the next trading day
    next_trading_days = nyse.valid_days(start_date=nyse_time.date() + timedelta(days=1), end_date=nyse_time.date() + timedelta(days=10))
    return next_trading_days[0].date()

In [123]:
get_current_or_next_trading_day()

datetime.date(2024, 10, 1)

In [124]:
def get_earnings():
    # Get the last full trading day and the current or next trading day
    last_trading_day = get_last_full_trading_day()
    next_trading_day = get_current_or_next_trading_day()

    # Define the SQL query with the specific dates and conditions
    query = f"""
    WITH LatestEarnings AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY `date` DESC) AS rn
        FROM `earnings_calendar`
        WHERE `when` IS NOT NULL
    )
    SELECT *
    FROM LatestEarnings
    WHERE rn = 1
    AND (
        (`date` = '{last_trading_day}' AND `when` = 'After market close') OR
        (`date` = '{next_trading_day}' AND `when` = 'Before market open')
    )
    ORDER BY `act_symbol` ASC;
    """
    
    # URL encode the query
    encoded_query = requests.utils.quote(query)
    
    # Set the DoltHub API endpoint and parameters
    endpoint = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/earnings/master?q={encoded_query}"
    
    # Make the API request
    response = requests.get(endpoint)
    response.raise_for_status()  # Check for errors

    # Check the content of the response
    data = response.json()

    # Handle the response and convert to a pandas DataFrame if successful
    if data.get('query_execution_status') == 'Success':
        df = pd.DataFrame(data['rows'])
        df = df.rename(columns={'act_symbol': 'symbol'})
        df = df[['symbol', 'date', 'when']]
        return df
    else:
        print(f"Query Error: {data.get('query_execution_message')}")

In [125]:
get_earnings()

,symbol,date,when
0,AYI,2024-10-01,Before market open
1,MKC,2024-10-01,Before market open
2,MKC.V,2024-10-01,Before market open
3,PAYX,2024-10-01,Before market open
4,POCI,2024-09-30,After market close
5,SUUN,2024-09-30,After market close
6,TRAK,2024-09-30,After market close
7,UNFI,2024-10-01,Before market open
8,VRAR,2024-09-30,After market close


In [126]:
def get_vol_data(symbols: list[str] = None, curated = True, include_yf = True):

    # Build the query to get the max date from the volatility_history table
    query_max_date = """
    SELECT MAX(`date`) AS max_date
    FROM `volatility_history`
    """
    encoded_query_max_date = requests.utils.quote(query_max_date)
    endpoint_max_date = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_max_date}"
    response_max_date = requests.get(endpoint_max_date) # Make the request to DoltHub for the max date
    response_max_date.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_max_date = response_max_date.json()

    # Extract the maximum date from the response
    if data_max_date.get('query_execution_status') == 'Success' and data_max_date.get('rows'):
        max_date = data_max_date['rows'][0]['max_date']
        print("Looking up vol data for:", max_date)
    else:
        print(f"Query Error: {data_max_date.get('query_execution_message')}")
        max_date = None

    if not max_date:
        return None
    
    if not symbols:
        earnings_table = get_earnings()
        symbols = earnings_table['symbol'].tolist()

    # Build the query to get the vol data for the max date
    symbols_str = ', '.join(f"'{symbol}'" for symbol in symbols)
    
    query_volatility = f"""
    SELECT *
    FROM `volatility_history`
    WHERE `date` = '{max_date}'
    AND `act_symbol` IN ({symbols_str})
    ORDER BY `act_symbol` ASC;
    """
    encoded_query_volatility = requests.utils.quote(query_volatility)
    endpoint_volatility = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_volatility}"
    response_volatility = requests.get(endpoint_volatility) # Make the request to DoltHub for volatility history
    response_volatility.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_volatility = response_volatility.json()

    # Check and convert the response to a pandas DataFrame
    if data_volatility.get('query_execution_status') == 'RowLimit':
        if data_volatility.get('rows'):
            df_volatility = pd.DataFrame(data_volatility['rows'])
        else:
            print("No data found for the maximum date.")
    else:
        df_volatility = pd.DataFrame(data_volatility['rows'])
    
    if not curated:
        return df_volatility
    
    df_vol = df_volatility[['act_symbol', 'date', 'hv_current', 'iv_current']].astype({'hv_current': 'float','iv_current': 'float'})
    df_vol['vol_premium'] = df_vol['iv_current']/df_vol['hv_current']

    if include_yf:
        end_date = datetime.now().date()
        start_date = end_date - timedelta(days=365*2)
        yf_data = yf.download(df_vol['act_symbol'].tolist(), start=start_date, end=end_date)
        
        # Calculate daily returns
        daily_returns = yf_data['Close'].pct_change()
        
        # Calculate 30-day rolling volatility
        rolling_volatility = daily_returns.rolling(window=30).std() * np.sqrt(252)
        
        # Get the most recent volatility for each symbol
        latest_volatility = rolling_volatility.iloc[-1]
        
        # Merge the calculated volatility and close price with df_vol
        df_vol = df_vol.merge(
            pd.DataFrame({
                'calculated_volatility': latest_volatility,
                'close': yf_data['Close'].iloc[-1]
            }),
            left_on='act_symbol',
            right_index=True
        )
        # Calculate vol_premium using the calculated volatility
        df_vol['vol_premium'] = df_vol['iv_current'] / df_vol['calculated_volatility']

    return df_vol.sort_values(by='vol_premium', ascending=False)



In [157]:
vol_df = get_vol_data()

Looking up vol data for: 2024-09-27


[*********************100%***********************]  5 of 5 completed


In [158]:
vol_df

,act_symbol,date,hv_current,iv_current,vol_premium,calculated_volatility,close
2,LW,2024-09-27,0.1716,0.5778,2.915869,0.198157,64.739998
1,CALM,2024-09-27,0.1888,0.3417,1.810462,0.188736,74.839996
4,RPM,2024-09-27,0.1952,0.2827,1.412074,0.200202,121.000000
3,NKE,2024-09-27,0.2534,0.3492,1.341805,0.260246,88.400002
0,CAG,2024-09-27,0.1933,0.2277,1.189284,0.191460,32.520000


In [136]:
def connect_to_IB(port=7497, clientid=0, symbol=None):
    util.startLoop()  # Needed in script mode
    ib = IB()
    try:
        ib.connect('127.0.0.1', port, clientId=clientid)
    except ConnectionError:
        ib = None  # Reset ib on failure
    return ib

In [137]:
ib = connect_to_IB()

In [186]:
ib.reqMarketDataType(2)


In [187]:
symbols = vol_df.act_symbol.tolist()

In [188]:
symbols

['LW', 'CALM', 'RPM', 'NKE', 'CAG']

In [189]:
symbol = symbols[3]
symbol

'NKE'

In [199]:
def get_filtered_put_options(ib, symbol, max_dte=60):
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(stock)
    
    # Get the stock price
    [ticker] = ib.reqTickers(stock)
    stock_price = ticker.marketPrice()
    
    if np.isnan(stock_price):
        print(f"Unable to get market price for {symbol}")
        return None
    
    # Calculate strike range
    lower_strike = stock_price * 0.85  # 20% below
    upper_strike = stock_price * 1.05  # 5% above
    
    # Get option chains
    chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next(c for c in chains if c.exchange == 'SMART')
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    
    # Filter strikes
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = ib.qualifyContracts(*contracts)
    
    # Request market data
    tickers = ib.reqTickers(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract
        })
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])
    
    return df, stock_price

# Usage example:
options_df, stock_price = get_filtered_put_options(ib, symbol)

Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=87.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=87.5, right='P', exchange='SMART')
Unknown contract: Option

In [200]:
options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
options_df['stock_price'] = stock_price
options_df['safety_margin'] = stock_price - options_df['Strike']




In [201]:
options_df[options_df['option_premium'] > 0.01].sort_values('safety_margin', ascending=False)

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
88,79.0,2024-11-08,38,0.85,3.0,1.66,157.0,"Option(conId=731946413, symbol='NKE', lastTrad...",0.010759,0.103348,83.63,4.63
100,80.0,2024-11-15,45,1.09,5.0,1.22,24.0,"Option(conId=705993053, symbol='NKE', lastTrad...",0.013625,0.110514,83.63,3.63
76,80.0,2024-11-01,31,0.90,1.0,1.09,333.0,"Option(conId=728253068, symbol='NKE', lastTrad...",0.011250,0.132460,83.63,3.63
61,81.0,2024-10-25,24,0.87,2416.0,1.11,1869.0,"Option(conId=727047673, symbol='NKE', lastTrad...",0.010741,0.163349,83.63,2.63
62,82.0,2024-10-25,24,1.01,229.0,1.49,3.0,"Option(conId=727047734, symbol='NKE', lastTrad...",0.012317,0.187322,83.63,1.63
26,82.0,2024-10-11,10,0.97,1.0,1.01,3.0,"Option(conId=725890603, symbol='NKE', lastTrad...",0.011829,0.431768,83.63,1.63
78,82.0,2024-11-01,31,1.10,813.0,2.20,1.0,"Option(conId=728253157, symbol='NKE', lastTrad...",0.013415,0.157946,83.63,1.63
44,82.0,2024-10-18,17,0.92,207.0,1.25,624.0,"Option(conId=730757699, symbol='NKE', lastTrad...",0.011220,0.240890,83.63,1.63
101,82.5,2024-11-15,45,1.62,211.0,1.85,9.0,"Option(conId=705993072, symbol='NKE', lastTrad...",0.019636,0.159273,83.63,1.13
45,82.5,2024-10-18,17,1.19,3.0,1.30,601.0,"Option(conId=694774765, symbol='NKE', lastTrad...",0.014424,0.309697,83.63,1.13


In [192]:
stk = Stock(symbol, exchange='SMART', currency='USD')
ib.qualifyContracts(stk)
[ticker] = ib.reqTickers(stk)

In [193]:
ticker

Ticker(contract=Stock(conId=10291, symbol='NKE', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='NKE', tradingClass='NKE'), time=datetime.datetime(2024, 10, 1, 21, 34, 8, 733068, tzinfo=datetime.timezone.utc), minTick=0.01, bid=83.56, bidSize=1.0, bidExchange='P', ask=83.7, askSize=6.0, askExchange='KT', last=83.51, lastSize=0.0, lastExchange='D', volume=192039.0, open=87.98, high=89.64, low=87.71, close=88.4, bboExchange='a60001', snapshotPermissions=3)

In [173]:
ticker

Ticker(contract=Stock(conId=272093, symbol='MSFT', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='MSFT', tradingClass='NMS'), minTick=0.01, bboExchange='9c0001', snapshotPermissions=4)

In [185]:
stk_price =ticker.marketPrice()
stk_price


66.0

In [147]:
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,ISE,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
1,NASDAQBX,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
2,BATS,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
3,PEARL,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
4,AMEX,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
5,GEMINI,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
6,SMART,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
7,MERCURY,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
8,MIAX,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."
9,CBOE,272942,PAYX,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90...."


In [148]:
chain = next(c for c in chains if c.tradingClass == stk.symbol and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='272942', tradingClass='PAYX', multiplier='100', expirations=['20241018', '20241115', '20241220', '20250117', '20250321', '20250620', '20250919', '20260116', '20270115'], strikes=[55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195.0])

In [149]:
strikes = [strike for strike in chain.strikes
        if stk_price - 20 < strike < stk_price + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P']

contracts = [Option(stk.symbol, expiration, strike, right, 'SMART', tradingClass=stk.symbol)
        for right in rights
        for expiration in expirations
        for strike in strikes]

In [150]:
contracts = ib.qualifyContracts(*contracts)


In [153]:
tickers = ib.reqTickers(*contracts)
df = util.df(tickers)
df

,contract,time,marketDataType,minTick,bid,bidSize,bidExchange,ask,askSize,askExchange,...,bidGreeks,askGreeks,lastGreeks,modelGreeks,auctionVolume,auctionPrice,auctionImbalance,regulatoryImbalance,bboExchange,snapshotPermissions
0,"Option(conId=715962855, symbol='PAYX', lastTra...",2024-10-01 20:21:13.111489+00:00,2,0.05,0.10,243.0,,0.15,340.0,,...,"(0, 0.29900005887401265, -0.02778980999238264,...","(0, 0.3207389876865678, -0.03712911936098058, ...","(0, 0.2989897642291784, -0.027786160545072578,...","(0, 0.3020403209972785, -0.156055198385585, 0....",NaN,NaN,NaN,NaN,c70003,0
1,"Option(conId=715962897, symbol='PAYX', lastTra...",2024-10-01 20:21:19.195355+00:00,2,0.05,0.20,355.0,,0.30,662.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3
2,"Option(conId=715962979, symbol='PAYX', lastTra...",2024-10-01 20:21:14.238486+00:00,2,0.05,0.55,989.0,,0.70,367.0,,...,"(0, 0.2091160689808409, -0.15999114147233856, ...","(0, 0.2222894966286629, -0.1740684274354359, 0...","(0, 0.21619769990917867, -0.16776543087371854,...","(0, 0.21718588644279566, -0.5057846126436618, ...",NaN,NaN,NaN,NaN,c70003,3
3,"Option(conId=715963040, symbol='PAYX', lastTra...",2024-10-01 20:21:19.195598+00:00,2,0.05,1.65,104.0,,1.85,158.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3
4,"Option(conId=715963096, symbol='PAYX', lastTra...",2024-10-01 20:21:19.195796+00:00,2,0.00,4.60,42.0,,4.90,66.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3
5,"Option(conId=715963167, symbol='PAYX', lastTra...",2024-10-01 20:21:14.238755+00:00,2,0.05,9.00,55.0,,10.20,37.0,,...,None,"(0, 0.3529537436122534, -0.7874227286028983, 1...",None,"(0, 0.18975111529731173, -0.968568262969464, 1...",NaN,NaN,NaN,NaN,c70003,3
6,"Option(conId=715963230, symbol='PAYX', lastTra...",2024-10-01 20:21:14.239073+00:00,2,0.05,12.60,51.0,,16.10,57.0,,...,None,"(0, 0.3309208180429977, -0.9154515668742079, 1...",None,"(0, 0.20654832649274713, -0.9999999999999984, ...",NaN,NaN,NaN,NaN,c70003,3
7,"Option(conId=715963282, symbol='PAYX', lastTra...",2024-10-01 20:21:19.197565+00:00,2,0.05,17.70,56.0,,21.30,101.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3
8,"Option(conId=715964996, symbol='PAYX', lastTra...",2024-10-01 20:21:19.197565+00:00,2,0.05,0.45,1.0,,0.60,513.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3
9,"Option(conId=715965047, symbol='PAYX', lastTra...",2024-10-01 20:21:19.197565+00:00,2,0.05,0.85,102.0,,1.05,251.0,,...,None,None,None,"(0, None, None, None, None, None, -2.0, -2.0, ...",NaN,NaN,NaN,NaN,c70003,3


In [154]:
df.iloc[0].contract

Option(conId=715962855, symbol='PAYX', lastTradeDateOrContractMonth='20241018', strike=125.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PAYX  241018P00125000', tradingClass='PAYX')

In [152]:
util.df(contracts)

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,description,issuerId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,OPT,715962855,PAYX,20241018,125.0,P,100,SMART,,USD,PAYX 241018P00125000,PAYX,False,,,,,,[],None
1,OPT,715962897,PAYX,20241018,130.0,P,100,SMART,,USD,PAYX 241018P00130000,PAYX,False,,,,,,[],None
2,OPT,715962979,PAYX,20241018,135.0,P,100,SMART,,USD,PAYX 241018P00135000,PAYX,False,,,,,,[],None
3,OPT,715963040,PAYX,20241018,140.0,P,100,SMART,,USD,PAYX 241018P00140000,PAYX,False,,,,,,[],None
4,OPT,715963096,PAYX,20241018,145.0,P,100,SMART,,USD,PAYX 241018P00145000,PAYX,False,,,,,,[],None
5,OPT,715963167,PAYX,20241018,150.0,P,100,SMART,,USD,PAYX 241018P00150000,PAYX,False,,,,,,[],None
6,OPT,715963230,PAYX,20241018,155.0,P,100,SMART,,USD,PAYX 241018P00155000,PAYX,False,,,,,,[],None
7,OPT,715963282,PAYX,20241018,160.0,P,100,SMART,,USD,PAYX 241018P00160000,PAYX,False,,,,,,[],None
8,OPT,715964996,PAYX,20241115,125.0,P,100,SMART,,USD,PAYX 241115P00125000,PAYX,False,,,,,,[],None
9,OPT,715965047,PAYX,20241115,130.0,P,100,SMART,,USD,PAYX 241115P00130000,PAYX,False,,,,,,[],None
